# 1. Preliminaries

#### 1.1 Preliminaries

In [1]:
# clear all variables
for i in list(globals().keys()):
    if(i[0] != '_'):
        exec('del {}'.format(i))

#suppress future warnings -- not really a good idea 
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

#%%  # clear all variables
#clear all variables
for i in list(globals().keys()):
    if(i[0] != '_'):
        exec('del {}'.format(i))
        
# suppress future warnings -- not really a good idea 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# ignore all warnings
import warnings
warnings.filterwarnings('ignore')

# import libraries/dependencies
import csv
import xlrd
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from matplotlib import figure

set_matplotlib_formats('png', 'pdf') # uses vector figures in pdf exports --
plt.style.use('seaborn-pastel')



KeyboardInterrupt



#### 1.2. Functions

In [ ]:
# some functions.. these will end up in a 'utils.py'
def load_data (fName):
    data = pd.read_csv(fName,header = 0)
    print('Data: ', data.shape)
    
    return data

# remove missing values with a given threshold
def remove_missing_values(data, nanThreshold):
    #1: remove  NaN: #  drop features that have over nanThreshold% missing values
    df=data.dropna(axis=1, thresh=int(nanThreshold*len(data)), subset=None, inplace=False)
    df.dropna(axis=0, inplace=True)
    print('before {} after {}'.format(data.shape, df.shape))   
    
    return df

# replace with LabenEncoder()
# encode features 
def encode_features_v2(df):
    # sex
    df=df.replace({'female':'0'}) 
    df=df.replace({'male':'1'})
  
    #drug_exposure
    df=df.replace({'No':'0'})
    df=df.replace({'Yes':'1'})
   
    #vr.stringent; vr.lenient
    df=df.replace({'non-responder':'0'})
    df=df.replace({'vasoresponder':'1'})

    # ec_right_ventricle
    df=df.replace({'normal':'0'})
    df=df.replace({'abnormal':'1'})

    # img_emphysema_category; img_fibrosis_category    
    df=df.replace({'none':'0'})
    df=df.replace({'minimal/mild':'1'})
    df=df.replace({'moderate':'2'})
    df=df.replace({'severe':'3'})

    # img_thromboembolic_disease
    #df=df.replace({'UNK':'NA'})
    df=df.replace({'no':'0'})
    df=df.replace({'yes':'1'})

    # smoking_habit
    df=df.replace({'no':'0'})
    df=df.replace({'past/current':'1'})
    
    # reveal_risk
    df=df.replace({'low risk':'0'})
    df=df.replace({'moderately high risk':'1'})
    df=df.replace({'high risk':'2'})
    df=df.replace({'very high risk':'3'})
    
    # initial_therapy
    df=df.replace({'PDE5':'0'})
    df=df.replace({'CCB':'1'})
    df=df.replace({'ERA':'2'})
    df=df.replace({'PA':'3'})
    df=df.replace({'sGC':'4'})
    df=df.replace({'combination therapy':'5'})
    
    #UNK -- NA
    # df=df.replace({'UNK':''})
    
    return df

# encode features 
def encode_features_v3(df):
    # sex
    df=df.replace({'female':'0'}) 
    df=df.replace({'male':'1'})
  
    #drug_exposure
    df=df.replace({'No':'0'})
    df=df.replace({'Yes':'1'})
   
    #vr.stringent; vr.lenient
    df=df.replace({'non-responder':'0'})
    df=df.replace({'vasoresponder':'1'})

    # ec_right_ventricle
    df=df.replace({'normal':'0'})
    df=df.replace({'abnormal':'1'})

    # img_emphysema_category; img_fibrosis_category    
    df=df.replace({'none':'0'})
    df=df.replace({'minimal/mild':'1'})
    df=df.replace({'moderate':'2'})
    df=df.replace({'severe':'3'})

    # img_thromboembolic_disease
    #df=df.replace({'UNK':'NA'})
    df=df.replace({'no':'0'})
    df=df.replace({'yes':'1'})

    # smoking_habit
    df=df.replace({'no':'0'})
    df=df.replace({'past/current':'1'})
    
    # reveal_risk
    df=df.replace({'low risk':'0'})
    df=df.replace({'moderately high risk':'1'})
    df=df.replace({'high risk':'2'})
    df=df.replace({'very high risk':'3'})
    
    # initial_therapy
    df=df.replace({'PDE5':'0'})
    df=df.replace({'CCB':'1'})
    df=df.replace({'ERA':'2'})
    df=df.replace({'PA':'3'})
    df=df.replace({'sGC':'4'})
    df=df.replace({'combination therapy':'5'})
    
    #UNK -- NA
    # df=df.replace({'UNK':''})
    
    return df


def create_folder(folder):
    import os     
    try: 
        os.mkdir(folder) 
    except FileExistsError:
        print("Directory [ %s ] already exists"%folder)

# remove low variance features
def remove_low_variance(df, thresh):
    from sklearn.feature_selection import VarianceThreshold
    target=df.clusters
    thresholder = VarianceThreshold(threshold=thresh)
    thresholder.fit_transform(df.drop('clusters',axis=1))
    selected_features_mask=thresholder.get_support()
    df = df.loc[:,thresholder.get_support(indices=False)] 
    df['clusters']=target
    print(df.shape)

    return df

def assess_missingness(data):

    # pandas series denoting features and the sum of their null values
    null_sum = data.isnull().sum()# instantiate columns for missing data
    total = null_sum.sort_values(ascending=False)
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False)
    
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    
    # drop rows that don't have any missing data; may comment out to omit
    df_NA = df_NA[ (df_NA.T != 0).any() ]
    
    return df_NA


'''
    Take input data 9training and testing
    Run Recursive Feature Elimination (RFE)
    based on 6 estimators -- SVR, RF, GBR, ETR, LR and XGB
'''
def run_RFE(X_train, y_train, X_test, y_test, num_to_select):
    from sklearn.feature_selection import RFE
    from sklearn.svm import SVR
  
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LogisticRegression
    from xgboost import XGBClassifier
    
    
    # classifiers
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import ExtraTreesClassifier
    from xgboost import XGBClassifier
    from sklearn.linear_model import LogisticRegression
    
    # classifiers
    np.random.seed(220)
    
    #features 
    features=X_test.columns
    
    # feature scaling
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    
    # estimators -- to get weights and rankings ... 
    svr = SVC(kernel='linear',random_state=220) # support vector machine
    rf = RandomForestClassifier(random_state=220)
    gbr = GradientBoostingClassifier(random_state=220,verbosity=0)
    logreg = LogisticRegression(random_state=220)
    xg = XGBClassifier(random_state=220)
    etr = ExtraTreesClassifier(random_state=220)
        
    # instantiate RFEs
    svr_rfe = RFE(svr, num_to_select, step=1, verbose=0)
    rf_rfe = RFE(rf, num_to_select, step=1, verbose=0)
    gbr_rfe = RFE(gbr, num_to_select, step=1, verbose=0)
    logreg_rfe = RFE(logreg, num_to_select, step=1, verbose=0)
    xg_rfe = RFE(xg, num_to_select, step=1, verbose=0)
    etr_rfe = RFE(etr, num_to_select, step=1, verbose=0)
    
    # train RFEs
    svr_rfe.fit(X_train,y_train); #print('fitting SVR')
    rf_rfe.fit(X_train,y_train); #print('fitting RandomforestRegressor')   
    gbr_rfe.fit(X_train,y_train); #print('fitting GradientBoostingRegressor')
    logreg_rfe.fit(X_train,y_train); #print('fitting LogisticRegression')
    xg_rfe.fit(X_train,y_train); #print('fitting XGBoost')
    etr_rfe.fit(X_train,y_train); #print('fitting ExtraTreesRegressor')
    
    # scores
    svr_score = svr_rfe.score(X_test,y_test)
    rf_score  = rf_rfe.score(X_test,y_test)
    gbr_score = gbr_rfe.score(X_test,y_test)
    logreg_score = logreg_rfe.score(X_test,y_test)
    xg_score = xg_rfe.score(X_test,y_test)
    etr_score = etr_rfe.score(X_test,y_test)
    
    # selected features
    rf_feature  = features[rf_rfe.support_]
    svr_feature  = features[svr_rfe.support_] # use
    gbr_feature = features[gbr_rfe.support_]
    logreg_feature = features[logreg_rfe.support_] 
    xg_feature = features[xg_rfe.support_]
    etr_feature = features[etr_rfe.support_]
    
    # feature rankings
    svr_ranking = svr_rfe.ranking_ # use
    rf_ranking  = rf_rfe.ranking_  # use
    gbr_ranking = gbr_rfe.ranking_ # use
    logreg_ranking = logreg_rfe.ranking_ # use
    xg_ranking  = xg_rfe.ranking_  # use
    etr_ranking = etr_rfe.ranking_ # use
    
    # selected features from all selectors
    selected_features=pd.DataFrame()
    selected_features['svr'] = pd.Series(svr_feature)
    selected_features['rf'] = pd.Series(rf_feature)
    selected_features['xg'] = pd.Series(xg_feature)
    selected_features['gbr'] = pd.Series(gbr_feature)
    selected_features['etr'] = pd.Series(etr_feature)
    selected_features['logreg'] = pd.Series(logreg_feature)
    
    # feature ranking from all selectors
    feature_ranking=pd.DataFrame()
    #feature_ranking['features']=pd.Series(features)
    feature_ranking['svr'] = pd.Series(svr_rfe.ranking_)
    feature_ranking['rf']  = pd.Series(rf_rfe.ranking_)
    feature_ranking['xg']  = pd.Series(xg_rfe.ranking_)
    feature_ranking['gbr'] = pd.Series(gbr_rfe.ranking_)
    feature_ranking['etr'] = pd.Series(etr_rfe.ranking_)
    feature_ranking['logreg'] = pd.Series(logreg_rfe.ranking_)
    
    # scores -- not used if only for feature ranking
    feature_ranking_scores=pd.DataFrame()
    feature_ranking_scores['svr'] = pd.Series(svr_score)
    feature_ranking_scores['rf']  = pd.Series(rf_score)
    feature_ranking_scores['xg']  = pd.Series(xg_score)
    feature_ranking_scores['gbr'] = pd.Series(gbr_score)
    feature_ranking_scores['etr'] = pd.Series(etr_score)
    feature_ranking_scores['logreg'] = pd.Series(logreg_score)
    
    return selected_features, feature_ranking, feature_ranking_scores

#### 1.3. Data Inspection

In [ ]:
# load data one dataset -- we have 50
df = pd.read_csv('G:/My Drive/ejammeh/Work/Projects/iPAH/Latest/data/imputed dataset 50/imputed_pheno_1.csv',header = 0)

# check missingness of raw data
#print(df.isnull().sum())

# inspect first few rows for every column
#print(df.head())

# Check variable types
#df.info()

# basic stats 
#df.describe()

# count cluster memberships
print(df.clusters.value_counts())

# inspect dataset
#df.head()

# 2: Data Pre-processing

#### 2.1. Discovery dataset

In [ ]:
#create folder to store results
create_folder('./results')
create_folder('./results/discovery_dataset')
create_folder('./results/discovery_dataset/preprocessing')
create_folder('./results/discovery_dataset/preprocessing/cleaned')
create_folder('./results/discovery_dataset/preprocessing/missingness')

# get duplicate featutes : these were identified from previous analysis of the datasets
duplicate_features = pd.read_csv('G:/My Drive/ejammeh/Work/Projects/iPAH/Latest/data/duplicate_features.csv', header=0).features

# bad features 
bad_features=np.array(pd.read_csv('G:/My Drive/ejammeh/Work/ipah_feature_selection/bad_features.csv').features)

for i in range (1,51):
    # 1: load datasets  
    exec("df = pd.read_csv('G:/My Drive/ejammeh/Work/Projects/iPAH/Latest/data/imputed dataset 50/imputed_pheno_%d.csv',header = 0)"%i)

    #2: remove duplicate features 
    df=df.drop(duplicate_features, axis=1)
    df=df.drop(columns=bad_features)

    # 3: only extract data for clusters A, B & E
    df = df[(df['clusters']=='A') | (df['clusters']=='B') | (df['clusters']=='E')]

    # 4: identify and remove features with >5% missing values
    missing = assess_missingness(df)
    missingness=pd.DataFrame(missing)
    
    '''
        4a: Plot missingness -- but only for features that has missing values
    '''
    fig=plt.figure(figsize=(15,5))
    sns.set_style("whitegrid", {'axes.grid' : False})
    ax=sns.barplot(data = missingness, x = missingness.index, y = 'Percent NA', ci = None, color='g')
    exec("plt.title('Discovery Dataset %d Missingness')"%i)
    plt.ylabel('Percentage of missing values')
    plt.xticks(rotation=90)
    plt.tick_params(labelsize=14)
    plt.tight_layout()
    
    exec("plt.savefig('./results/discovery_dataset/preprocessing/missingness/data_%d_missingness.png')"%i)
    exec("missingness.to_csv('./results/discovery_dataset/preprocessing/missingness/missingness_%d.csv',index=False)"%i)
    
    # no need to dislay all figures
    if i > 1:
        plt.close(fig)

    # 5: drop those features over 5% missingness
    features_to_drop = pd.Series(missingness[missingness['Percent NA'] > 5].index) # any feature with 5% missingness to be dropped
    df=df.drop(columns = list(features_to_drop)).drop('Unnamed: 0',axis=1)   

    # remove some dodgy features
    df = df.drop(['id', 'cohort_group', 'diagnosis_verified', 'case', 'reveal_risk', 'cum.haz'], axis=1)
    
    # 6: encode datasets
    df = encode_features_v2(df); 

    # 7: drop missing values 
    df=df.dropna()
    exec("ds_%d"%i+" =df.copy()")
    
    # 8: save to file
    exec("df.to_csv('./results/discovery_dataset/preprocessing/cleaned/cleaned_discovery_dataset_%d.csv',index=False)"%i)

# sanity check
df.shape
#df.info()


#### 2.2. Validation dataset

In [ ]:
#create folder to store results
create_folder('./results')
create_folder('./results/validation_dataset')
create_folder('./results/validation_dataset/preprocessing')
create_folder('./results/validation_dataset/preprocessing/cleaned')
create_folder('./results/validation_dataset/preprocessing/missingness')

# get duplicate featutes : these were identified from previous analysis of the datasets
duplicate_features = pd.read_csv('G:/My Drive/ejammeh/Work/Projects/iPAH/Latest/data/duplicate_features.csv', header=0).features

for i in range (1,51):
    # 1: load datasets  
    exec("df = pd.read_csv('G:/My Drive/ejammeh/Work/Projects/iPAH/Latest/data/imputed validation dataset 50/validation_imputed_pheno_%d.csv',header = 0)"%i)

    #2: remove duplicate features 
    df=df.drop(duplicate_features, axis=1)

    # 4: identify and remove features with >5% missing values
    missing = assess_missingness(df)
    missingness=pd.DataFrame(missing)
    
    '''
        4a: Plot missingness -- but only for features that has missing values
    '''
    fig=plt.figure(figsize=(15,5))
    sns.set_style("whitegrid", {'axes.grid' : False})
    ax=sns.barplot(data = missingness, x = missingness.index, y = 'Percent NA', ci = None, color='g')
    exec("plt.title('Discovery Dataset %d Missingness')"%i)
    plt.ylabel('Percentage of missing values')
    plt.xticks(rotation=90)
    plt.tick_params(labelsize=14)
    plt.tight_layout()
    
    exec("plt.savefig('./results/validation_dataset/preprocessing/missingness/data_%d_missingness.png')"%i)
    exec("missingness.to_csv('./results/validation_dataset/preprocessing/missingness/missingness_%d.csv')"%i)
    
    # no need to dislay all figures
    if i > 1:
        plt.close(fig)

    # 5: drop those features over 5% missingness
    features_to_drop = pd.Series(missingness[missingness['Percent NA'] > 5].index) # any feature with 5% missingness to be dropped
    df=df.drop(columns = list(features_to_drop)).drop('Unnamed: 0',axis=1)   

    # remove some dodgy features
    #df = df.drop(['id', 'cohort_group', 'diagnosis_verified', 'case', 'reveal_risk'], axis=1)
    #df = df.drop(['reveal_risk'], axis=1)
    
    # 6: encode datasets
    df = encode_features_v2(df)

    # 7: drop missing values 
    df = df.dropna()
    exec("dv_%d"%i+"=df.copy()")
    
    # 8: save to file
    exec("df.to_csv('./results/validation_dataset/preprocessing/cleaned/cleaned_validation_dataset_%d.csv',index=False)"%i)
    
# sanity check
df.shape
#df.info()


# 3. Robust feature selection & ranking 

#### 3.1. Setup

In [ ]:
# Import Libraries; create folders; create datasets for each cluster; and setup experiment
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.utils import resample
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.svm import SVC
import statistics as stats
from sklearn import metrics
from random import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
import math

# create folders to store results
create_folder('./results')
create_folder('./results/feature_selection')
create_folder('./results/feature_selection/cluster_A')
create_folder('./results/feature_selection/cluster_B')
create_folder('./results/feature_selection/cluster_E')

create_folder('./results/feature_selection/feature_ranking')
create_folder('./results/feature_selection/feature_ranking/cluster_A')
create_folder('./results/feature_selection/feature_ranking/cluster_B')
create_folder('./results/feature_selection/feature_ranking/cluster_E')

#number of samples
n_repeat = 1000 # number of resampling
n_bstraps = 500 # number of bootstraps
n_features = 21

'''
    1: Load the pre-processed and cleaned dataset
    2: Create sub datasets around Cluster A, B & E
    ** start with just one dataset before scaling to all datasets
'''
# this should be run for each discovery dataset -- and results aggregrated i.e. the feature rankings
ds_1=pd.read_csv('./results/discovery_dataset/preprocessing/cleaned/cleaned_discovery_dataset_1.csv',header=0)
df_A=ds_1.copy(); df_A.clusters=df_A.clusters.replace({'A':1, 'B':0, 'C':0, 'D':0, 'E':0})
df_B=ds_1.copy(); df_B.clusters=df_B.clusters.replace({'A':0, 'B':1, 'C':0, 'D':0, 'E':0})
df_E=ds_1.copy(); df_E.clusters=df_E.clusters.replace({'A':0, 'B':0, 'C':0, 'D':0, 'E':1})


 #### 3.2. Cluster A feature selection & rankings

In [ ]:

# should add an outer for loop for clusters

# Go through Cluter A
for size in range (1, n_features): 
    for repeat in range(1,n_repeat, 1): 
        
        # grab rankings of each feature based on svr and rfe 
        exec("svr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("rf_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("gbr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("etr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("logreg_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("xg_ranking_%d"%size+"_repeat_%d = []"%repeat)
 
        # Resampling using 90% of original dataset
        sample = resample(df_A, replace=False, n_samples=math.ceil(df_A.shape[0]*0.9), random_state=int((random()*100)))  
        
        # bootstrap level
        for bs in range(1, n_bstraps): 
            '''
                using resample with replacement -- fuul set/size
                thus some samples may be repeated
            '''
            boot = resample(sample, replace=True, n_samples=sample.shape[0], random_state=int((random()*100))) 
           
            '''
                extract data and target/clusters
                then split dataset into training and testing datasets 70:30
            '''
            X = boot.drop(columns='clusters') # data
            target = boot.clusters #class
            X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, stratify=target, random_state=220)
            
            '''
                Then run Recursive Feature Elimination to select a set of features
                we use LinearSVC and RandomForestClassifier
                
            '''
            # rfe feature selection -- a number of estimators -- number of features to select is set to one -- just a feature ranking process
            selected_features, feature_ranking, feature_ranking_scores = run_RFE(X_train, y_train, X_test, y_test, size)
            
            if bs==1:
                exec("svr_ranking_%d"%size+"_repeat_%d = feature_ranking.svr"%repeat)
                exec("rf_ranking_%d"%size+"_repeat_%d = feature_ranking.rf"%repeat)
                exec("xg_ranking_%d"%size+"_repeat_%d = feature_ranking.xg"%repeat)
                exec("gbr_ranking_%d"%size+"_repeat_%d = feature_ranking.gbr"%repeat)
                exec("etr_ranking_%d"%size+"_repeat_%d = feature_ranking.etr"%repeat)
                exec("logreg_ranking_%d"%size+"_repeat_%d = feature_ranking.logreg"%repeat)
            elif bs>1:
                exec("svr_ranking_%d"%size+"_repeat_%d += feature_ranking.svr"%repeat)
                exec("rf_ranking_%d"%size+"_repeat_%d += feature_ranking.rf"%repeat)
                exec("xg_ranking_%d"%size+"_repeat_%d += feature_ranking.xg"%repeat)
                exec("gbr_ranking_%d"%size+"_repeat_%d += feature_ranking.gbr"%repeat)
                exec("etr_ranking_%d"%size+"_repeat_%d += feature_ranking.etr"%repeat)
                exec("logreg_ranking_%d"%size+"_repeat_%d += feature_ranking.logreg"%repeat)
            
            print("Robust feature selection: signature size {} sample {} bootstrap {} ".format(size, repeat, bs), end = '\r')

        # aggregrate ranking from all rankers over all bootstrap subsamples
        #exec("rankings = svr_ranking_%d"%size+"_repeat_%d"%repeat+"+rf_ranking_%d"%size+"_repeat_%d"%repeat+" + xg_ranking_%d"%size+"_repeat_%d"%repeat+" +gbr_ranking_%d"%size+"_repeat_%d"%repeat+"+etr_ranking_%d"%size+"_repeat_%d"%repeat+"+ logreg_ranking_%d"%size+"_repeat_%d"%repeat)     
        #exec("rfe_size_%d_signatures"%size+"['rankings_%d']"%repeat+"= rankings")     
             
# get aggregrate results
for size in range (1, n_features): 
    for repeat in range(1,n_repeat, 1): 
        for algo in ['svr','rf','xg','gbr','etr','logreg']:
            if repeat==1:
                exec("overall_%s"%algo+"_size_%d"%size+"_ranking = %s"%algo+"_ranking_%d"%size+"_repeat_%d"%repeat)
            else:
                exec("overall_%s"%algo+"_size_%d"%size+"_ranking += %s"%algo+"_ranking_%d"%size+"_repeat_%d"%repeat)
    #final rankings
    exec("final_size_%d"%size+"_rankings=pd.DataFrame()")
    exec("final_size_%d"%size+"_rankings['features']=pd.Series(X.columns)")
    exec("final_size_%d"%size+"_rankings['svr']=overall_svr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['rf']=overall_rf_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['xg']=overall_xg_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['gbr']=overall_gbr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['etr']=overall_etr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['logreg']=overall_logreg_size_%d"%size+"_ranking")
    

# create folders to store results
create_folder('./results')
create_folder('./results/feature_selection')
create_folder('./results/feature_selection/feature_ranking')
create_folder('./results/feature_selection/feature_ranking/cluster_A')
create_folder('./results/feature_selection/feature_ranking/cluster_B')
create_folder('./results/feature_selection/feature_ranking/cluster_E')

for size in range (1, n_features): 
    # get features
    exec("size_%d"%size+"_rankings = pd.DataFrame()")
    exec("size_%d"%size+"_rankings ['features']=pd.Series(X.columns)")
    exec("size_%d"%size+"_rankings ['svr']=final_size_%d"%size+"_rankings.svr")
    exec("size_%d"%size+"_rankings ['rf']=final_size_%d"%size+"_rankings.rf")
    exec("size_%d"%size+"_rankings ['xg']=final_size_%d"%size+"_rankings.xg")
    exec("size_%d"%size+"_rankings ['gbr']=final_size_%d"%size+"_rankings.gbr")
    exec("size_%d"%size+"_rankings ['etr']=final_size_%d"%size+"_rankings.etr")
    exec("size_%d"%size+"_rankings ['logreg']=final_size_%d"%size+"_rankings.logreg")
    exec("size_%d"%size+"_rankings ['average']=size_%d"%size+"_rankings.mean(axis=1)")
    exec("size_%d"%size+"_rankings.sort_values(by='svr', ascending=True, inplace=True)")

    # save to file
    exec("size_%d"%size+"_rankings.to_csv('./results/feature_selection/feature_ranking/cluster_A/cluster_A_size_%d"%size+"_feature_rankings_svr.csv')")


#### 3.3. Cluster B feature selection & rankings

In [ ]:

# should add an outer for loop for clusters

# Go through Cluter B
for size in range (1, n_features): 
    
    for repeat in range(1,n_repeat, 1): 
        
        # grab rankings of each feature based on svr and rfe 
        exec("svr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("rf_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("gbr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("etr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("logreg_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("xg_ranking_%d"%size+"_repeat_%d = []"%repeat)

        
        # Resampling using 90% of original dataset
        sample = resample(df_B, replace=False, n_samples=math.ceil(df_B.shape[0]*0.9), random_state=int((random()*100)))  
        
        # bootstrap level
        for bs in range(1, n_bstraps): 
            '''
                using resample with replacement -- fuul set/size
                thus some samples may be repeated
            '''
            boot = resample(sample, replace=True, n_samples=sample.shape[0], random_state=int((random()*100))) 
           
            '''
                extract data and target/clusters
                then split dataset into training and testing datasets 70:30
            '''
            X = boot.drop(columns='clusters') # data
            target = boot.clusters #class
            X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, stratify=target, random_state=220)
            
            '''
                Then run Recursive Feature Elimination to select a set of features
                we use LinearSVC and RandomForestClassifier
                
            '''
            # rfe feature selection -- a number of estimators -- number of features to select is set to one -- just a feature ranking process
            selected_features, feature_ranking, feature_ranking_scores = run_RFE(X_train, y_train, X_test, y_test, size)
            
            if bs==1:
                exec("svr_ranking_%d"%size+"_repeat_%d = feature_ranking.svr"%repeat)
                exec("rf_ranking_%d"%size+"_repeat_%d = feature_ranking.rf"%repeat)
                exec("xg_ranking_%d"%size+"_repeat_%d = feature_ranking.xg"%repeat)
                exec("gbr_ranking_%d"%size+"_repeat_%d = feature_ranking.gbr"%repeat)
                exec("etr_ranking_%d"%size+"_repeat_%d = feature_ranking.etr"%repeat)
                exec("logreg_ranking_%d"%size+"_repeat_%d = feature_ranking.logreg"%repeat)
            elif bs>1:
                exec("svr_ranking_%d"%size+"_repeat_%d += feature_ranking.svr"%repeat)
                exec("rf_ranking_%d"%size+"_repeat_%d += feature_ranking.rf"%repeat)
                exec("xg_ranking_%d"%size+"_repeat_%d += feature_ranking.xg"%repeat)
                exec("gbr_ranking_%d"%size+"_repeat_%d += feature_ranking.gbr"%repeat)
                exec("etr_ranking_%d"%size+"_repeat_%d += feature_ranking.etr"%repeat)
                exec("logreg_ranking_%d"%size+"_repeat_%d += feature_ranking.logreg"%repeat)
            
            print("Cluster B -- Robust feature selection: signature size {} sample {} bootstrap {} ".format(size, repeat, bs), end = '\r')

        # aggregrate ranking from all rankers over all bootstrap subsamples
        #exec("rankings = svr_ranking_%d"%size+"_repeat_%d"%repeat+"+rf_ranking_%d"%size+"_repeat_%d"%repeat+" + xg_ranking_%d"%size+"_repeat_%d"%repeat+" +gbr_ranking_%d"%size+"_repeat_%d"%repeat+"+etr_ranking_%d"%size+"_repeat_%d"%repeat+"+ logreg_ranking_%d"%size+"_repeat_%d"%repeat)     
        #exec("rfe_size_%d_signatures"%size+"['rankings_%d']"%repeat+"= rankings")     

             
# get aggregrate results
for size in range (1, n_features): 
    for repeat in range(1,n_repeat, 1): 
        for algo in ['svr','rf','xg','gbr','etr','logreg']:
            if repeat==1:
                exec("overall_%s"%algo+"_size_%d"%size+"_ranking = %s"%algo+"_ranking_%d"%size+"_repeat_%d"%repeat)
            else:
                exec("overall_%s"%algo+"_size_%d"%size+"_ranking += %s"%algo+"_ranking_%d"%size+"_repeat_%d"%repeat)
    #final rankings
    exec("final_size_%d"%size+"_rankings=pd.DataFrame()")
    exec("final_size_%d"%size+"_rankings['features']=pd.Series(X.columns)")
    exec("final_size_%d"%size+"_rankings['svr']=overall_svr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['rf']=overall_rf_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['xg']=overall_xg_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['gbr']=overall_gbr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['etr']=overall_etr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['logreg']=overall_logreg_size_%d"%size+"_ranking")
    

for size in range (1, n_features): 
    # get features
    exec("size_%d"%size+"_rankings = pd.DataFrame()")
    exec("size_%d"%size+"_rankings ['features']=pd.Series(X.columns)")
    exec("size_%d"%size+"_rankings ['svr']=final_size_%d"%size+"_rankings.svr")
    exec("size_%d"%size+"_rankings ['rf']=final_size_%d"%size+"_rankings.rf")
    exec("size_%d"%size+"_rankings ['xg']=final_size_%d"%size+"_rankings.xg")
    exec("size_%d"%size+"_rankings ['gbr']=final_size_%d"%size+"_rankings.gbr")
    exec("size_%d"%size+"_rankings ['etr']=final_size_%d"%size+"_rankings.etr")
    exec("size_%d"%size+"_rankings ['logreg']=final_size_%d"%size+"_rankings.logreg")
    exec("size_%d"%size+"_rankings ['average']=size_%d"%size+"_rankings.mean(axis=1)")
    exec("size_%d"%size+"_rankings.sort_values(by='svr', ascending=True, inplace=True)")

    # save to file
    exec("size_%d"%size+"_rankings.to_csv('./results/feature_selection/feature_ranking/cluster_B/cluster_B_size_%d"%size+"_feature_rankings_svr.csv')")


#### 3.4. Cluster E feature selection & rankings

In [ ]:

# should add an outer for loop for clusters

# Go through Cluter E
for size in range (1, n_features): 
    for repeat in range(1,n_repeat, 1): 
        
        # grab rankings of each feature based on svr and rfe 
        exec("svr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("rf_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("gbr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("etr_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("logreg_ranking_%d"%size+"_repeat_%d = []"%repeat)
        exec("xg_ranking_%d"%size+"_repeat_%d = []"%repeat)

        
        # Resampling using 90% of original dataset
        sample = resample(df_E, replace=False, n_samples=math.ceil(df_E.shape[0]*0.9), random_state=int((random()*100)))  
        
        # bootstrap level
        for bs in range(1, n_bstraps): 
            '''
                using resample with replacement -- fuul set/size
                thus some samples may be repeated
            '''
            boot = resample(sample, replace=True, n_samples=sample.shape[0], random_state=int((random()*100))) 
           
            '''
                extract data and target/clusters
                then split dataset into training and testing datasets 70:30
            '''
            X = boot.drop(columns='clusters') # data
            target = boot.clusters #class
            X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, stratify=target, random_state=220)
            
            '''
                Then run Recursive Feature Elimination to select a set of features
                we use LinearSVC and RandomForestClassifier
                
            '''
            # rfe feature selection -- a number of estimators -- number of features to select is set to one -- just a feature ranking process
            selected_features, feature_ranking, feature_ranking_scores = run_RFE(X_train, y_train, X_test, y_test, size)
            
            if bs==1:
                exec("svr_ranking_%d"%size+"_repeat_%d = feature_ranking.svr"%repeat)
                exec("rf_ranking_%d"%size+"_repeat_%d = feature_ranking.rf"%repeat)
                exec("xg_ranking_%d"%size+"_repeat_%d = feature_ranking.xg"%repeat)
                exec("gbr_ranking_%d"%size+"_repeat_%d = feature_ranking.gbr"%repeat)
                exec("etr_ranking_%d"%size+"_repeat_%d = feature_ranking.etr"%repeat)
                exec("logreg_ranking_%d"%size+"_repeat_%d = feature_ranking.logreg"%repeat)
            elif bs>1:
                exec("svr_ranking_%d"%size+"_repeat_%d += feature_ranking.svr"%repeat)
                exec("rf_ranking_%d"%size+"_repeat_%d += feature_ranking.rf"%repeat)
                exec("xg_ranking_%d"%size+"_repeat_%d += feature_ranking.xg"%repeat)
                exec("gbr_ranking_%d"%size+"_repeat_%d += feature_ranking.gbr"%repeat)
                exec("etr_ranking_%d"%size+"_repeat_%d += feature_ranking.etr"%repeat)
                exec("logreg_ranking_%d"%size+"_repeat_%d += feature_ranking.logreg"%repeat)
            
            print("Cluster E -- Robust feature selection: signature size {} sample {} bootstrap {} ".format(size, repeat, bs), end = '\r')

        # aggregrate ranking from all rankers over all bootstrap subsamples
        #exec("rankings = svr_ranking_%d"%size+"_repeat_%d"%repeat+"+rf_ranking_%d"%size+"_repeat_%d"%repeat+" + xg_ranking_%d"%size+"_repeat_%d"%repeat+" +gbr_ranking_%d"%size+"_repeat_%d"%repeat+"+etr_ranking_%d"%size+"_repeat_%d"%repeat+"+ logreg_ranking_%d"%size+"_repeat_%d"%repeat)     
        #exec("rfe_size_%d_signatures"%size+"['rankings_%d']"%repeat+"= rankings")     
         
# get aggregrate results
for size in range (1, n_features): 
    for repeat in range(1,n_repeat, 1): 
        for algo in ['svr','rf','xg','gbr','etr','logreg']:
            if repeat==1:
                exec("overall_%s"%algo+"_size_%d"%size+"_ranking = %s"%algo+"_ranking_%d"%size+"_repeat_%d"%repeat)
            else:
                exec("overall_%s"%algo+"_size_%d"%size+"_ranking += %s"%algo+"_ranking_%d"%size+"_repeat_%d"%repeat)
    #final rankings
    exec("final_size_%d"%size+"_rankings=pd.DataFrame()")
    exec("final_size_%d"%size+"_rankings['features']=pd.Series(X.columns)")
    exec("final_size_%d"%size+"_rankings['svr']=overall_svr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['rf']=overall_rf_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['xg']=overall_xg_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['gbr']=overall_gbr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['etr']=overall_etr_size_%d"%size+"_ranking")
    exec("final_size_%d"%size+"_rankings['logreg']=overall_logreg_size_%d"%size+"_ranking")
    
for size in range (1, n_features): 
    # get features
    exec("size_%d"%size+"_rankings = pd.DataFrame()")
    exec("size_%d"%size+"_rankings ['features']=pd.Series(X.columns)")
    exec("size_%d"%size+"_rankings ['svr']=final_size_%d"%size+"_rankings.svr")
    exec("size_%d"%size+"_rankings ['rf']=final_size_%d"%size+"_rankings.rf")
    exec("size_%d"%size+"_rankings ['xg']=final_size_%d"%size+"_rankings.xg")
    exec("size_%d"%size+"_rankings ['gbr']=final_size_%d"%size+"_rankings.gbr")
    exec("size_%d"%size+"_rankings ['etr']=final_size_%d"%size+"_rankings.etr")
    exec("size_%d"%size+"_rankings ['logreg']=final_size_%d"%size+"_rankings.logreg")
    exec("size_%d"%size+"_rankings ['average']=size_%d"%size+"_rankings.mean(axis=1)")
    exec("size_%d"%size+"_rankings.sort_values(by='svr', ascending=True, inplace=True)")

    # save to file
    exec("size_%d"%size+"_rankings.to_csv('./results/feature_selection/feature_ranking/cluster_E/cluster_E_size_%d"%size+"_feature_rankings_svr.csv')")


## 4. Classification

#### 4.1. Cluster A classification

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier 

# Use signature A for classification
for algo in ['svr', 'svc', 'lasso', 'lr', 'dt', 'knn', 'lda', 'gnb', 'rf']:
    exec("%s"%algo+"_train_accy = []")
    exec("%s"%algo+"_test_accy = []")

for size in range (1, n_features): 
    # 1: get signature
    #exec("features=pd.read_csv('G:/My Drive/ejammeh/Work/ipah_feature_selection/robust_feature_selection/code/results/feature_selection/feature_ranking/cluster_A/cluster_A_size_%d"%size+"_feature_rankings_svr.csv',header=0).features[0:%d]"%size)
    exec("tmp=pd.read_csv('G:/My Drive/ejammeh/Work/ipah_feature_selection/robust_feature_selection/code/results/feature_selection/feature_ranking/cluster_A/cluster_A_size_%d"%size+"_feature_rankings_svr.csv',header=0)")
    tmp.sort_values(by='average', ascending=True, inplace=True)
    exec("features=tmp.features[0:%d]"%size)
    
    # 2: filter data based on selected signature
    X=df_A[features]
    y=df_A.clusters
    
    # 3: Split the dataset into  Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 220)
    
    # 4: data scaling
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    #X_test = scaler.transform(X_test)
    
    # 5: classifiers creation and fitting
    svr = LinearSVC(random_state=220).fit(X_train, y_train)  # Linear SVC
    svc = SVC(kernel='rbf',random_state=220).fit(X_train, y_train)  # SCV
    lasso = Lasso(alpha=1e-6,normalize=True, max_iter=1e5, random_state=220).fit(X_train, y_train) # Lasso
    lr = LogisticRegression(n_jobs = -1, random_state=220).fit(X_train, y_train) # Log
    dt = DecisionTreeClassifier(random_state=220).fit(X_train, y_train)
    knn = KNeighborsClassifier().fit(X_train, y_train)
    lda = LinearDiscriminantAnalysis().fit(X_train, y_train)
    gnb = GaussianNB().fit(X_train, y_train)
    rf = RandomForestClassifier(random_state=220).fit(X_train, y_train)
    
    # classifiers able to get feature weights -- linear
    for algo in ['svr', 'lasso', 'lr']:
        exec("%s"%algo+"_weights=pd.Series(np.ravel(%s"%algo+".coef_))")
        exec("%s"%algo+"_weights = %s"%algo+"_weights/np.max(abs(%s"%algo+"_weights))")
        exec("%s"%algo+"_train_pred = %s"%algo+".predict(X_train)")
        exec("%s"%algo+"_test_pred = %s"%algo+".predict(X_test)")
        exec("%s"%algo+"_test_acc = round(%s"%algo+".score(X_test, y_test) * 100, 2)")
        exec("%s"%algo+"_train_acc = round(%s"%algo+".score(X_train, y_train) * 100, 2)")           
        exec("%s"%algo+"_train_accy.append(%s"%algo+"_train_acc)")
        exec("%s"%algo+"_test_accy.append(%s"%algo+"_test_acc)")
    
    # classifiers unable to get feature weights 
    for algo in ['svc', 'dt', 'knn', 'lda', 'gnb', 'rf']:
        exec("%s"%algo+"_train_pred = %s"%algo+".predict(X_train)")
        exec("%s"%algo+"_test_pred = %s"%algo+".predict(X_test)")
        exec("%s"%algo+"_test_acc = round(%s"%algo+".score(X_test, y_test) * 100, 2)")
        exec("%s"%algo+"_train_acc = round(%s"%algo+".score(X_train, y_train) * 100, 2)")           
        exec("%s"%algo+"_train_accy.append(%s"%algo+"_train_acc)")
        exec("%s"%algo+"_test_accy.append(%s"%algo+"_test_acc)")
    
# save results
cluster_A_train_clf_res=pd.DataFrame()
cluster_A_train_clf_res['size']=range(1,21)
for algo in ['svr', 'lr', 'knn', 'rf']:
    exec("cluster_A_train_clf_res['%s"%algo+"']=%s"%algo+"_train_accy")
    
# save results
cluster_A_test_clf_res=pd.DataFrame()
cluster_A_test_clf_res['size']=range(1,21)
for algo in ['svr', 'lr', 'knn', 'rf']:
    exec("cluster_A_test_clf_res['%s"%algo+"']=%s"%algo+"_test_accy")
    

#### 4.2. Cluster B classification

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier 

# Use signature B for classification
for algo in ['svr', 'svc', 'lasso', 'lr', 'dt', 'knn', 'lda', 'gnb', 'rf']:
    exec("%s"%algo+"_train_accy = []")
    exec("%s"%algo+"_test_accy = []")

for size in range (1, n_features): 
    # 1: get signature
    #exec("features=pd.read_csv('G:/My Drive/ejammeh/Work/ipah_feature_selection/robust_feature_selection/code/results/feature_selection/feature_ranking/cluster_A/cluster_A_size_%d"%size+"_feature_rankings_svr.csv',header=0).features[0:%d]"%size)
    exec("tmp=pd.read_csv('G:/My Drive/ejammeh/Work/ipah_feature_selection/robust_feature_selection/code/results/feature_selection/feature_ranking/cluster_B/cluster_B_size_%d"%size+"_feature_rankings_svr.csv',header=0)")
    tmp.sort_values(by='average', ascending=True, inplace=True)
    exec("features=tmp.features[0:%d]"%size)
    
    # 2: filter data based on selected signature
    X=df_B[features]
    y=df_B.clusters
    
    # 3: Split the dataset into  Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 220)
    
    # 4: data scaling
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    #X_test = scaler.transform(X_test)
    
    # 5: classifiers creation and fitting
    svr = LinearSVC(random_state=220).fit(X_train, y_train)  # Linear SVC
    svc = SVC(kernel='rbf',random_state=220).fit(X_train, y_train)  # SCV
    lasso = Lasso(alpha=1e-6,normalize=True, max_iter=1e5, random_state=220).fit(X_train, y_train) # Lasso
    lr = LogisticRegression(n_jobs = -1, random_state=220).fit(X_train, y_train) # Log
    dt = DecisionTreeClassifier(random_state=220).fit(X_train, y_train)
    knn = KNeighborsClassifier().fit(X_train, y_train)
    lda = LinearDiscriminantAnalysis().fit(X_train, y_train)
    gnb = GaussianNB().fit(X_train, y_train)
    rf = RandomForestClassifier(random_state=220).fit(X_train, y_train)
    
    # classifiers able to get feature weights -- linear
    for algo in ['svr', 'lasso', 'lr']:
        exec("%s"%algo+"_weights=pd.Series(np.ravel(%s"%algo+".coef_))")
        exec("%s"%algo+"_weights = %s"%algo+"_weights/np.max(abs(%s"%algo+"_weights))")
        exec("%s"%algo+"_train_pred = %s"%algo+".predict(X_train)")
        exec("%s"%algo+"_test_pred = %s"%algo+".predict(X_test)")
        exec("%s"%algo+"_test_acc = round(%s"%algo+".score(X_test, y_test) * 100, 2)")
        exec("%s"%algo+"_train_acc = round(%s"%algo+".score(X_train, y_train) * 100, 2)")           
        exec("%s"%algo+"_train_accy.append(%s"%algo+"_train_acc)")
        exec("%s"%algo+"_test_accy.append(%s"%algo+"_test_acc)")
    
    # classifiers unable to get feature weights 
    for algo in ['svc', 'dt', 'knn', 'lda', 'gnb', 'rf']:
        exec("%s"%algo+"_train_pred = %s"%algo+".predict(X_train)")
        exec("%s"%algo+"_test_pred = %s"%algo+".predict(X_test)")
        exec("%s"%algo+"_test_acc = round(%s"%algo+".score(X_test, y_test) * 100, 2)")
        exec("%s"%algo+"_train_acc = round(%s"%algo+".score(X_train, y_train) * 100, 2)")           
        exec("%s"%algo+"_train_accy.append(%s"%algo+"_train_acc)")
        exec("%s"%algo+"_test_accy.append(%s"%algo+"_test_acc)")
    
# save results
cluster_B_train_clf_res=pd.DataFrame()
cluster_B_train_clf_res['size']=range(1,21)
for algo in ['svr', 'lr', 'knn', 'rf']:
    exec("cluster_B_train_clf_res['%s"%algo+"']=%s"%algo+"_train_accy")
    
# save results
cluster_B_test_clf_res=pd.DataFrame()
cluster_B_test_clf_res['size']=range(1,21)
for algo in ['svr', 'lr', 'knn', 'rf']:
    exec("cluster_B_test_clf_res['%s"%algo+"']=%s"%algo+"_test_accy")
    

#### 4.3. Cluster E classification

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier 

# Use signature B for classification
for algo in ['svr', 'svc', 'lasso', 'lr', 'dt', 'knn', 'lda', 'gnb', 'rf']:
    exec("%s"%algo+"_train_accy = []")
    exec("%s"%algo+"_test_accy = []")

for size in range (1, n_features): 
    # 1: get signature
    #exec("features=pd.read_csv('G:/My Drive/ejammeh/Work/ipah_feature_selection/robust_feature_selection/code/results/feature_selection/feature_ranking/cluster_A/cluster_A_size_%d"%size+"_feature_rankings_svr.csv',header=0).features[0:%d]"%size)
    exec("tmp=pd.read_csv('G:/My Drive/ejammeh/Work/ipah_feature_selection/robust_feature_selection/code/results/feature_selection/feature_ranking/cluster_E/cluster_E_size_%d"%size+"_feature_rankings_svr.csv',header=0)")
    tmp.sort_values(by='average', ascending=True, inplace=True)
    exec("features=tmp.features[0:%d]"%size)
    
    # 2: filter data based on selected signature
    X=df_E[features]
    y=df_E.clusters
    
    # 3: Split the dataset into  Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 220)
    
    # 4: data scaling
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # 5: classifiers creation and fitting
    svr = LinearSVC(random_state=220).fit(X_train, y_train)  # Linear SVC
    svc = SVC(kernel='rbf',random_state=220).fit(X_train, y_train)  # SCV
    lasso = Lasso(alpha=1e-6,normalize=True, max_iter=1e5, random_state=220).fit(X_train, y_train) # Lasso
    lr = LogisticRegression(n_jobs = -1, random_state=220).fit(X_train, y_train) # Log
    dt = DecisionTreeClassifier(random_state=220).fit(X_train, y_train)
    knn = KNeighborsClassifier().fit(X_train, y_train)
    lda = LinearDiscriminantAnalysis().fit(X_train, y_train)
    gnb = GaussianNB().fit(X_train, y_train)
    rf = RandomForestClassifier(random_state=220).fit(X_train, y_train)
    
    # classifiers able to get feature weights -- linear
    for algo in ['svr', 'lasso', 'lr']:
        exec("%s"%algo+"_weights=pd.Series(np.ravel(%s"%algo+".coef_))")
        exec("%s"%algo+"_weights = %s"%algo+"_weights/np.max(abs(%s"%algo+"_weights))")
        exec("%s"%algo+"_train_pred = %s"%algo+".predict(X_train)")
        exec("%s"%algo+"_test_pred = %s"%algo+".predict(X_test)")
        exec("%s"%algo+"_test_acc = round(%s"%algo+".score(X_test, y_test) * 100, 2)")
        exec("%s"%algo+"_train_acc = round(%s"%algo+".score(X_train, y_train) * 100, 2)")           
        exec("%s"%algo+"_train_accy.append(%s"%algo+"_train_acc)")
        exec("%s"%algo+"_test_accy.append(%s"%algo+"_test_acc)")
    
    # classifiers unable to get feature weights 
    for algo in ['svc', 'dt', 'knn', 'lda', 'gnb', 'rf']:
        exec("%s"%algo+"_train_pred = %s"%algo+".predict(X_train)")
        exec("%s"%algo+"_test_pred = %s"%algo+".predict(X_test)")
        exec("%s"%algo+"_test_acc = round(%s"%algo+".score(X_test, y_test) * 100, 2)")
        exec("%s"%algo+"_train_acc = round(%s"%algo+".score(X_train, y_train) * 100, 2)")           
        exec("%s"%algo+"_train_accy.append(%s"%algo+"_train_acc)")
        exec("%s"%algo+"_test_accy.append(%s"%algo+"_test_acc)")
    
# save results
cluster_E_train_clf_res=pd.DataFrame()
cluster_E_train_clf_res['size']=range(1,21)
for algo in ['svr', 'lr', 'knn', 'rf']:
    exec("cluster_E_train_clf_res['%s"%algo+"']=%s"%algo+"_train_accy")
    
# save results
cluster_E_test_clf_res=pd.DataFrame()
cluster_E_test_clf_res['size']=range(1,21)
for algo in ['svr', 'lr', 'knn', 'rf']:
    exec("cluster_E_test_clf_res['%s"%algo+"']=%s"%algo+"_test_accy")
    